# Getting started

**Adapted from**: SOSTOOLS' SOSDEMO1 (See Section 4.1 of [SOSTOOLS User's Manual](http://sysos.eng.ox.ac.uk/sostools/sostools.pdf)) and Example 2.4 of [PJ08]

P. Parrilo and A. Jadbabaie
*Approximation of the joint spectral radius using sum of squares*.
Linear Algebra and its Applications, Elsevier (2008), 428, 2385-2402

In [1]:
using DynamicPolynomials
@polyvar x y
p = 2*x^4 + 2*x^3*y - x^2*y^2 + 5*y^4

5y⁴ - x²y² + 2x³y + 2x⁴

We need to pick an SDP solver, see [here](https://jump.dev/JuMP.jl/v1.12/installation/#Supported-solvers) for a list of the available choices.
We use `SOSModel` instead of `Model` to be able to use the `>=` syntax for Sum-of-Squares constraints.

In [2]:
using SumOfSquares
import CSDP
solver = optimizer_with_attributes(CSDP.Optimizer, MOI.Silent() => true)
model = SOSModel(solver)
con_ref = @constraint(model, p >= 0)
optimize!(model)
primal_status(model)

FEASIBLE_POINT::ResultStatusCode = 1

We see above that the solver found a feasible solution.
We now inspect this solution:

In [3]:
q = gram_matrix(con_ref)

GramMatrix with row/column basis:
 MonomialBasis([y^2, x*y, x^2])
And entries in a 3×3 SymMatrix{Float64}:
  5.0                     4.4408920985006264e-17  -2.9518518518518517
  4.4408920985006264e-17  4.903703703703703        1.0
 -2.9518518518518517      1.0                      2.0

We can get the SOS decomposition from the gram matrix as follows:

In [4]:
sosdec = SOSDecomposition(q)

(-2.1236350925148186*y^2 + 0.685602733872361*x*y + 1.4043287476933908*x^2)^2 + (-0.6931517747987985*y^2 - 2.105348694339436*x*y - 0.02034325143230109*x^2)^2 + (0.0985627258797928*y^2 - 0.034050994900062616*x*y + 0.16567112157245356*x^2)^2

We now seek for the SOS decomposition of the following polynomial:

In [5]:
p = 4*x^4*y^6 + x^2 - x*y^2 + y^2

y² + x² - xy² + 4x⁴y⁶

We build the same model as previously with this new polynomial.
Here we can use `Model` instead of `SOSModel` as we explicitly constrain
`p` to belong to the SOS cone with `p in SOSCone()`.

In [6]:
model = Model(solver)
con_ref = @constraint(model, p in SOSCone())
optimize!(model)
primal_status(model)

FEASIBLE_POINT::ResultStatusCode = 1

We can query the SOS decomposition directly from the constraint reference
as follows:

In [7]:
sos_decomposition(con_ref)

(-0.8633544111907666*y + 0.19297758710767485*x*y + 3.0514841249382637e-18*x*y^2 + 1.974804063246114*x^2*y^3)^2 + (-1.8744116508952374e-16*y + 0.7962855107812752*x - 1.0528501085696115e-16*x*y - 1.7452642425911875*x*y^2 - 8.635065971616514e-17*x^2*y^3)^2 + (0.24383463418983603*y - 3.527832841584653e-16*x - 1.5486392128975743*x*y + 3.741600489401517e-17*x*y^2 + 0.2579336224358899*x^2*y^3)^2 + (-5.3930412033044254e-17*y + 0.6049209744419549*x - 7.96731030959412e-17*x*y + 0.27599821010522496*x*y^2 - 9.028384658229797e-18*x^2*y^3)^2 + (-0.4417735074073056*y - 5.4253712206747063e-17*x - 0.10009637589813161*x*y + 1.4688082190158866e-17*x*y^2 - 0.1833552786361387*x^2*y^3)^2

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*